In [1]:
import sys
import torch
import soundfile as sf 
sys.path.append('/home/oneran/Wave-Split-CSU2021/测试代码/newConvTasNet')
from model.pretrainedDPTConvTasNet import ConvTasNet
import numpy as np
from asteroid.models import BaseModel
from pretrained_sudormrf import get_fusion_model
import time
torch.set_num_threads(7)

In [ ]:
parameters = np.load('/home/oneran/Wave-Split-CSU2021/models/parameters.npy', allow_pickle=True).item()
net = ConvTasNet(parameters["N"], parameters["L"], parameters["B"], parameters["Sc"], parameters["H"],
                    parameters["P"],
                    parameters["X"], parameters["R"], parameters["C"], parameters["N_ED"], parameters["mask_act"],
                    parameters["attention"]).cuda()

optim = torch.optim.Adam(net.parameters(), lr=parameters["LR"])
checkpoint = torch.load('/home/oneran/Wave-Split-CSU2021/models/ckpt.pth')

init_epoch = checkpoint["epoch"]
net.load_state_dict(checkpoint['model_state_dict'])
net.cpu()
pass

In [21]:
net = get_fusion_model()
ckpt = torch.load('/home/oneran/Wave-Split-CSU2021/models/ckpt.pth')
best = torch.load('/home/oneran/Wave-Split-CSU2021/models/best.pth')
net.load_state_dict(best)
net.cpu()
pass

/home/oneran/sudo_rm_rf/sudo_rm_rf/dnn/models/improved_sudormrf.py:255: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.encoder.weight)
/home/oneran/sudo_rm_rf/sudo_rm_rf/dnn/models/improved_sudormrf.py:283: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.decoder.weight)


In [ ]:
inputs = torch.ones(1, 1, 64000)
# traced_module = torch.jit.trace(net, inputs)
# traced_module.save('/home/oneran/Wave-Split-CSU2021/模型压缩/DPT_ConvTasNet.pt')
traced_module.eval()
net.eval()
torch.save(net, '/home/oneran/Wave-Split-CSU2021/模型压缩/DPT_ConvTasNet.pth')
pass

In [36]:
root_path = '/home/oneran/Wave-Split-CSU2021/测试数据/16k'
net.eval()
def f(root_path, wavfile):
    wav, _ = sf.read(root_path + '/' + wavfile)
    pad_ = 80 - len(wav) % 80 + 1
    wav = np.concatenate([wav, np.zeros((pad_))], axis=-1)
    ret = net(torch.Tensor(wav.reshape(1, 1, -1).astype(np.float32))).detach().numpy()[0]
    sf.write(root_path + '/' + '{}_est1.wav'.format(wavfile[:-4]), ret[0] / np.max(ret) / 2, 16000)
    sf.write(root_path + '/' + '{}_est2.wav'.format(wavfile[:-4]), ret[1] / np.max(ret) / 2, 16000)

In [38]:
net.cpu()
f(root_path, 'test_2.wav')

In [ ]:
model = BaseModel.from_pretrained("JorisCos/ConvTasNet_Libri2Mix_sepnoisy_16k")
model.sample_rate = 16000
model.separate('/home/oneran/Wave-Split-CSU2021/测试数据/16k/test_4.wav', force_overwrite=True)

In [ ]:
net.eval()
model.eval()
_in = torch.rand((1, 1, 16000 * 8))
time_1 = time.time()
net(_in)
time_2 = time.time()
model(_in)
time_3 = time.time()
print(time_3-time_2, time_2-time_1)